In [116]:
# Template

#[Webpage] → Extract → [Raw DF]
#          ↓
#       Clean → [Clean DF]
#          ↓
#      Convert → [USD, GBP, EUR, INR]
#          ↓
#       Save → CSV Table

In [117]:
#Logging Codes
    
from datetime import datetime
LOGFILE = "code_log.txt"

def log_progress(message: str) -> None:
    """
    Append a timestamped log message to code_log.txt.
    """
    timestamp = datetime.now().strftime("date:%Y-%m-%d time:%H-%M-%S")
    with open(LOGFILE, "a") as logfile:
            logfile.write(f"{timestamp} - {message}\n")
    
log_progress("Test log line")

#Example logs
#log_progress("Extraction Started")
#log_progress("Extraction Completed")    
#log_progress("Cleanse & Normalize Process Started")
#log_progress("Cleanse & Normalize Completed")
#log_progress("Data Transformation Started")
#log_progress("Data Transformation Completed")
#log_progress("Currency Conversion Started")
#log_progress("Currency Conversion Completed")
#log_progress("DataBase Connection Initiated")
#log_progress("Loading Transformed DataFrame into SQL Started")
#log_progress("Loading Transformed DataFrame into SQL Completed")
#log_progress("Query Started")
#log_progress("Query Completed")
#log_progress("DataBase Connection Completed. Connection Closed.")
#log_progress("Logging Completed")
#log_progress("Logging Verified in code_log.txt")

#You will write these log entries in the code_log.txt file using the log_progress() function at each significant step

In [118]:
# ETL Pipeline

#Constants
local_html = "Largest_Banks_Wiki_Local.html"
log_file = "code_log.txt"

#Exract
from bs4 import BeautifulSoup
import pandas as pd
import io

log_progress("Extraction Started")

with open("Largest_Banks_Wiki_Local.html", encoding="utf-8") as f:
    html = f.read()

    soup = BeautifulSoup(html, "html.parser")

headline_span = soup.find("span", id="By_market_capitalization")

heading = headline_span.find_parent(["h2", "h3"])
bank_table = heading.find_next("table")

df = pd.read_html(io.StringIO(str(bank_table)))[0]

log_progress("Extraction Completed")

print(df.head())

   Rank                                Bank name  Market cap (US$ billion)
0     1                           JPMorgan Chase                    432.92
1     2                          Bank of America                    231.52
2     3  Industrial and Commercial Bank of China                    194.56
3     4               Agricultural Bank of China                    160.68
4     5                                HDFC Bank                    157.91


In [119]:
# ETL Pipeline

# Clean & Normalize
log_progress("Cleanse & Normalize Process Started")

# Keep only the top 10 rows
df = df.head(10)[["Bank name", "Market cap (US$ billion)"]].copy()
df.columns = ["Name", "MC_USD_Billion"]
print(df)

log_progress("Cleanse & Normalize Completed")

                                      Name  MC_USD_Billion
0                           JPMorgan Chase          432.92
1                          Bank of America          231.52
2  Industrial and Commercial Bank of China          194.56
3               Agricultural Bank of China          160.68
4                                HDFC Bank          157.91
5                              Wells Fargo          155.87
6                        HSBC Holdings PLC          148.90
7                           Morgan Stanley          140.83
8                  China Construction Bank          139.82
9                            Bank of China          136.81


In [120]:
from bs4 import BeautifulSoup
import io

local_html = "Largest_Banks_Wiki_Local.html"

def extract(local_html_path):
    log_progress("Extraction Started")

    with open(local_html_path, encoding="utf-8") as f:
        html = f.read()

    soup = BeautifulSoup(html, "html.parser")

    # Find heading "By market capitalization" and the next table
    headline_span = soup.find("span", id="By_market_capitalization")
    heading = headline_span.find_parent(["h2", "h3"])
    bank_table = heading.find_next("table")

    df = pd.read_html(io.StringIO(str(bank_table)))[0]

    # keep only top 10 and the two relevant columns
    df = df.head(10)[["Bank name", "Market cap (US$ billion)"]].copy()
    df.columns = ["Name", "MC_USD_Billion"]

    log_progress("Extraction Completed")
    return df

In [121]:
# ETL Pipeline

#Transform
import pandas as pd

exchange_rate_path = "exchange_rate.csv"

def transform(df, exchange_rate_path):
    """
    Add MC_GBP_Billion, MC_EUR_Billion, MC_INR_Billion
    using rates from exchange_rate.csv.
    """
    log_progress("Data Transformation Started")

    # Read the exchange rates file
    rates = pd.read_csv(exchange_rate_path)
    print(rates)  # run once to confirm column names

    # Expecting something like:
    # Currency,Rate
    # GBP,0.8
    # EUR,0.93
    # INR,82.95

    rate_dict = dict(zip(rates["Currency"], rates["Rate"]))

    # Create new columns by multiplying USD by the rate
    df["MC_GBP_Billion"] = (df["MC_USD_Billion"] * rate_dict["GBP"]).round(2)
    df["MC_EUR_Billion"] = (df["MC_USD_Billion"] * rate_dict["EUR"]).round(2)
    df["MC_INR_Billion"] = (df["MC_USD_Billion"] * rate_dict["INR"]).round(2)

    log_progress("Data Transformation Completed")
    return df


In [122]:
df=extract(local_html)
print(df)

                                      Name  MC_USD_Billion
0                           JPMorgan Chase          432.92
1                          Bank of America          231.52
2  Industrial and Commercial Bank of China          194.56
3               Agricultural Bank of China          160.68
4                                HDFC Bank          157.91
5                              Wells Fargo          155.87
6                        HSBC Holdings PLC          148.90
7                           Morgan Stanley          140.83
8                  China Construction Bank          139.82
9                            Bank of China          136.81


In [123]:
df = transform(df, exchange_rate_path)
print(df)

  Currency   Rate
0      EUR   0.93
1      GBP   0.80
2      INR  82.95
                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                        HSBC Holdings PLC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71

In [124]:
# ETL Pipeline

#Save to CSV
def load_to_csv(df, output_path):
    """
    Save the transformed DataFrame to a local CSV file.
    """
    log_progress("Saving Data to CSV Started")
    df.to_csv(output_path, index=False)
    log_progress("Saving Data to CSV Completed")

In [125]:
output_csv_path = "Largest_banks_data.csv"

load_to_csv(df, output_csv_path)
print("CSV saved!")

CSV saved!


In [126]:
# ETL Pipeline

#Load to SQL

import sqlite3

db_name = "Banks.db"
table_name = "Largest_banks"

def load_to_db(df, db_name, table_name):
    """
    Load the transformed DataFrame into a SQLite database table.
    """
    log_progress("DB Load Started")

    # Connect (creates Banks.db if it doesn't exist)
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Create table if it doesn't exist
    cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            Name TEXT,
            MC_USD_Billion REAL,
            MC_GBP_Billion REAL,
            MC_EUR_Billion REAL,
            MC_INR_Billion REAL
        )
    """)

    # Load data (replace any existing contents of the table)
    df.to_sql(table_name, conn, if_exists="replace", index=False)

    conn.commit()
    conn.close()

    log_progress("DB Load Completed")

In [127]:
load_to_db(df, db_name, table_name)
print("Data loaded into database!")

Data loaded into database!


In [128]:
import pandas as pd
import sqlite3

conn = sqlite3.connect(db_name)
check_df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
conn.close()

print(check_df)
log_progress("DB connection closed")

                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                        HSBC Holdings PLC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2          180.94        16138.75

In [129]:
import pandas as pd
import sqlite3

conn = sqlite3.connect(db_name)
query1 = pd.read_sql(f"SELECT * FROM {table_name}", conn)
conn.close()

print(query1)
log_progress("Query: SELECT * Verified")

                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                        HSBC Holdings PLC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2          180.94        16138.75

In [130]:
conn = sqlite3.connect(db_name)
query2 = pd.read_sql(f"""
    SELECT Name, MC_USD_Billion 
    FROM {table_name}
    ORDER BY MC_USD_Billion DESC
    LIMIT 1
""", conn)
conn.close()

print(query2)
log_progress("Query: Highest Market Cap Verified")

             Name  MC_USD_Billion
0  JPMorgan Chase          432.92


In [132]:
import pandas as pd
import sqlite3

def run_queries(db_name, table_name):
    """
    Execute verification SQL queries on the loaded ETL data.
    """
    log_progress("SQL Queries Started")

    conn = sqlite3.connect(db_name)

    # 1) Full table
    query_all = pd.read_sql(f"SELECT * FROM {table_name}", conn)
    print("🔎 FULL TABLE:\n", query_all, "\n")
    log_progress("Query: SELECT * Verified")

    # 2) Highest market cap bank in USD
    query_max = pd.read_sql(f"""
        SELECT Name, MC_USD_Billion 
        FROM {table_name}
        ORDER BY MC_USD_Billion DESC
        LIMIT 1
    """, conn)
    print("💰 HIGHEST MARKET CAP (USD):\n", query_max, "\n")
    log_progress("Query: Highest Market Cap Verified")

    # 3) Average market caps in each currency
    query_avg = pd.read_sql(f"""
        SELECT
            AVG(MC_USD_Billion) AS Avg_USD,
            AVG(MC_GBP_Billion) AS Avg_GBP,
            AVG(MC_EUR_Billion) AS Avg_EUR,
            AVG(MC_INR_Billion) AS Avg_INR
        FROM {table_name}
    """, conn)
    print("📊 AVERAGE MARKET CAPS:\n", query_avg, "\n")
    log_progress("Query: Averages Verified")

    conn.close()
    log_progress("SQL Queries Completed")

    return query_all, query_max, query_avg

In [133]:
run_queries(db_name, table_name)

🔎 FULL TABLE:
                                       Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                        HSBC Holdings PLC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2          180.94 

(                                      Name  MC_USD_Billion  MC_GBP_Billion  \
 0                           JPMorgan Chase          432.92          346.34   
 1                          Bank of America          231.52          185.22   
 2  Industrial and Commercial Bank of China          194.56          155.65   
 3               Agricultural Bank of China          160.68          128.54   
 4                                HDFC Bank          157.91          126.33   
 5                              Wells Fargo          155.87          124.70   
 6                        HSBC Holdings PLC          148.90          119.12   
 7                           Morgan Stanley          140.83          112.66   
 8                  China Construction Bank          139.82          111.86   
 9                            Bank of China          136.81          109.45   
 
    MC_EUR_Billion  MC_INR_Billion  
 0          402.62        35910.71  
 1          215.31        19204.58  
 2          180.94

In [134]:
log_progress("Logging Completed")
log_progress("Logging Verified in 'code_log.txt'")
log_progress("ETL Pipeline Execution Completed")